### Building a Softmax Classifier for Images in PyTorch
Adapted from https://machinelearningmastery.com/building-a-softmax-classifier-for-images-in-pytorch/

In [1]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torch import nn, optim
import os

# Path to your dataset
data_dir = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/ma-boston/buildings'

# Define transformations: resize, convert to tensor, normalize
transform = transforms.Compose([
    transforms.Resize((28, 28)),  # Resize to 28x28 to match input size of Softmax model
    transforms.Grayscale(),  # Ensure single channel
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]
])

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split the dataset into train and validation sets
train_size = int(0.875 * len(dataset))  # 3500 out of 4000 images
val_size = len(dataset) - train_size    # 500 images for validation
train_data, val_data = random_split(dataset, [train_size, val_size])

# Build custom softmax module
class Softmax(nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super(Softmax, self).__init__()
        self.linear = nn.Linear(n_inputs, n_outputs)

    def forward(self, x):
        pred = self.linear(x)
        return pred

# Get number of classes
num_classes = len(dataset.classes)

# Call Softmax Classifier
model_softmax = Softmax(28*28, num_classes)  # Input size is 28x28, outputs depend on the number of classes
model_softmax.state_dict()

# Define loss, optimizer, and dataloader for train and validation sets
optimizer = optim.SGD(model_softmax.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
batch_size = 16
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)

epochs = 200
Loss = []
acc = []

# Training and validation loops
for epoch in range(epochs):
    model_softmax.train()  # Set model to training mode
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model_softmax(images.view(-1, 28*28))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model_softmax.eval()  # Set model to evaluation mode
    correct = 0
    with torch.no_grad():  # No gradient computation needed during evaluation
        for images, labels in val_loader:
            outputs = model_softmax(images.view(-1, 28*28))
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / len(val_data)
    acc.append(accuracy)
    Loss.append(loss.item())
    if epoch % 10 == 0:
        print(f'Epoch: {epoch}. Loss: {loss.item():.4f}. Accuracy: {accuracy:.2f}%')

# Plotting Loss
plt.plot(Loss)
plt.xlabel("No. of epochs")
plt.ylabel("Total loss")
plt.title("Training Loss")
plt.show()

# Plotting Accuracy
plt.plot(acc)
plt.xlabel("No. of epochs")
plt.ylabel("Total accuracy")
plt.title("Validation Accuracy")
plt.show()

FileNotFoundError: Couldn't find any class folder in /Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/ma-boston/buildings.